<a href="https://colab.research.google.com/github/EricLBuehler/Automatic-Differentiation-Custom/blob/main/autodiff.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Automatic Differentiation

Automatic Differentiation is a core tool used to calculate derivatives which are key to machine learning. This is my personal implementation.

Eric Buehler 2023

In [154]:
# AD
#https://towardsdatascience.com/build-your-own-automatic-differentiation-program-6ecd585eec2a
#https://e-dorigatti.github.io/math/deep%20learning/2020/04/07/autodiff.html
#https://jingnanshi.com/blog/autodiff.html
#https://github.com/karpathy/micrograd

# Softmax derivative
#https://aimatters.wordpress.com/2019/06/17/the-softmax-function-derivative/

In [155]:
from abc import ABC, abstractmethod
from typing import *
from functools import reduce
import numpy as np

Define a base abstract class for all values, and a general OperatorLike class.

In [156]:
class DifferentiableValue(ABC):
    count = 0
    def __init__(self):
        DifferentiableValue.count += 1
        self.id = DifferentiableValue.count 

    @abstractmethod
    def backward(self, var):
        pass

    @abstractmethod
    def forward(self):
        pass

    @abstractmethod
    def __repr__(self) -> str:
        pass

class OperatorLike(ABC):
    pass

General function

- `generate_topo` topographically sorts a graph, ensuring a directed acyclic graph.

In [157]:
def generate_topo(graph: DifferentiableValue) -> List[DifferentiableValue]:
    topo = []
    visited = set()
    def build_topo(node: DifferentiableValue) -> List[DifferentiableValue]:
        if node not in visited:
            visited.add(node)
            if hasattr(node, "inputs"):
                for input in node.inputs:
                    build_topo(input)
            topo.append(node)
        return topo
    return build_topo(graph)

Define a graph to hold the operations, and a gradient graph to hold gradients.

In [158]:
class GradientGraph:
    def __init__(self, graph: List[DifferentiableValue]):
        self.graph = graph
    
    def wrt(self, var: DifferentiableValue) -> np.ndarray:
        if var not in self.graph:
            raise KeyError(f"Variable {var} not found in gradient graph.")
        for v in self.graph:
            if v == var:
                return v.gradient

    def __repr__(self):
        return "GradientGraph: {}".format(", ".join([str(item) for item in self.graph]))

class Graph:
    def __init__(self):
        self.values = []
        global _graph
        _graph = self
        self.has_backwarded = False
        self.clean_graph = lambda values: (values := values[:-1])

    def __enter__(self):
        return self
    
    def __exit__(self, exc_type, exc_value, traceback):
        pass

    def forward(self):
        self.clean_graph(self.values)
        return self.values[-1].forward()

    def backward(self):
        self.clean_graph(self.values)
        graph = self.values.copy()
        res = self.values[-1].backward()
        self.values = graph
        self.has_backwarded = True
        return res
    
    def create_gradient_graph(self) -> GradientGraph:
        assert self.backward

        graph = self.values
        filtered = set()
        
        for item in graph:
            topo = generate_topo(item)
            for value in topo:
                if not (isinstance(value, OperatorLike) and isinstance(value, Variable)):
                    filtered.add(value)
                
        return GradientGraph(list(filtered))

    def __repr__(self):
        return "Graph: {}".format(", ".join([str(item) for item in self.values]))

Define constant and variable values.

The `.backward` functions return the derivative of constants and variables.

In [159]:
class Constant(DifferentiableValue):
    count = 0

    def __init__(self, value: Union[SupportsFloat, np.ndarray]):
        super().__init__()
        self.value = np.array([value]) if isinstance(value, SupportsFloat) else value
        Constant.count += 1
        
        self.gradient = 0
        
    def backward(self):
        self.gradient = 0
                
        return _graph
        
    def forward(self) -> Any:
        return self.value

    def __repr__(self) -> str:
        return f"Constant({self.value}, g={self.gradient})"

class Variable(DifferentiableValue):
    count = 0

    def __init__(self, value: Union[SupportsFloat, np.ndarray] = None, name = None):
        super().__init__()
        self._value = np.array([value]) if isinstance(value, SupportsFloat) else value
        self.name = name
        Variable.count += 1
        
        self.gradient = 0
        
    def backward(self):
        self.gradient = 1
                
        return self
    
    @property
    def value(self):
        return self._value
    
    @value.setter
    def value(self, value):
        if self.value == None:
            raise ValueError("Variable does not have value")
        self._value = value
        
    def forward(self) -> np.ndarray:
        if self.value == None:
            raise ValueError("Variable does not have value")
        return self.value

    def __repr__(self) -> str:
        return f"Variable('{self.name}' {self.value}, g={self.gradient})"

Define operation "nodes" that act like values.

The `.backward` functions return the effective value after applying the chain rule, for sums, products, and powers.

See https://jingnanshi.com/blog/autodiff.html for a great explanation.
Note that the equations above table 1 show how the gradients are accumulated, like can be seen in the `_backward` functions.

In [160]:
class Sum(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [left, right]
        Sum.count += 1
        
        self.gradient = 0

        def _backward():
            for input in self.inputs:
                input.gradient += self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return sum([input.forward() for input in self.inputs])
    
    def __repr__(self) -> str:
        return "Sum({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Product(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, left: DifferentiableValue, right:  DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [left, right]
        Product.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += self.inputs[1].forward() * self.gradient
            self.inputs[1].gradient += self.inputs[0].forward() * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([reduce((lambda x, y: x * y), [input.forward() for input in self.inputs])])

    def __repr__(self) -> str:
        return "Product({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Power(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, base: DifferentiableValue, pow:  DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [base, pow]
        Power.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += (self.inputs[1].forward() * self.inputs[0].forward() ** (self.inputs[1] - 1).forward()) * self.gradient
            self.inputs[1].gradient += np.log(self.inputs[0].forward()) * self.inputs[0].forward() ** (self.inputs[1].forward()) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([reduce((lambda x, y: x ** y), [input.forward() for input in self.inputs])])

    def __repr__(self) -> str:
        return "Power({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

Trigonometric functions

In [161]:
class Sine(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        Sine.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += np.cos(self.inputs[0].forward()) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([np.sin(self.inputs[0].forward())])

    def __repr__(self) -> str:
        return "Sine({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Cosine(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        Cosine.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += -np.sin(self.inputs[0].forward()) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([np.cos(self.inputs[0].forward())])

    def __repr__(self) -> str:
        return "Cosine({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Tangent(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        Tangent.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += (1 / (np.cos(self.inputs[0].forward())**2)) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.tan(self.inputs[0].forward())

    def __repr__(self) -> str:
        return "Tangent({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

Activation Functions

In [162]:
class ReLU(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        Tangent.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += (1 if self.inputs[0].forward() > 0 else 0) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([max(0, self.inputs[0].forward())])

    def __repr__(self) -> str:
        return "ReLU({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class LeakyReLU(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue, negative_slope: SupportsFloat = 0.01):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        self.negative_slope = negative_slope
        Tangent.count += 1
        
        self.gradient = 0

        def _backward():
            self.inputs[0].gradient += (1 if self.inputs[0].forward() >= 0 else self.negative_slope) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
        
    def forward(self) -> np.ndarray:
        return np.array([self.inputs[0].forward() if self.inputs[0].forward() >= 0 else self.inputs[0].forward() * self.negative_slope])

    def __repr__(self) -> str:
        return "LeakyReLU({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Sigmoid(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, x: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = [x]
        Tangent.count += 1
        
        self.gradient = 0

        def _backward():
            x = self.inputs[0].forward()
            self.inputs[0].gradient += (self.sigmoid(x) * (1-self.sigmoid(x))) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self
    
    @staticmethod
    def sigmoid(x: np.ndarray) -> np.ndarray:
        return 1 / (1 + np.exp(-x))
        
    def forward(self) -> np.ndarray:
        return self.sigmoid(self.inputs[0].forward())

    def __repr__(self) -> str:
        return "Sigmoid({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

class Softmax(DifferentiableValue, OperatorLike):
    count = 0

    def __init__(self, *inputs: DifferentiableValue):
        super().__init__()
        _graph.values.append(self)
        self.inputs = inputs
        Tangent.count += 1
        
        self.gradient = 0

        def _backward():
            softmax = self.softmax()
            softmax_vector = softmax.reshape(softmax.shape[0],1)
            softmax_matrix = np.tile(softmax_vector,softmax.shape[0])
            self.inputs[0].gradient += (np.diag(softmax) - (softmax_matrix * np.transpose(softmax_matrix))) * self.gradient

        self._backward = _backward
        
    def backward(self):
        topo = generate_topo(self)
        
        self.gradient = 1
        for node in reversed(topo):
            if hasattr(node, "_backward"):
                node._backward()
                
        return self

    def softmax(self) -> np.ndarray:
        denom = sum([np.exp(input.forward()) for input in self.inputs])
        return np.array([np.exp(input.forward()) / denom for input in self.inputs])
        
    def forward(self) -> np.ndarray:
        denom = sum([np.exp(input.forward()) for input in self.inputs])
        return np.array([np.exp(input.forward()) / denom for input in self.inputs])

    def __repr__(self) -> str:
        return "Softmax({}, g={})".format(', '.join([str(input) for input in self.inputs]), self.gradient)

Define a `generate_operation` function to act as a closure and preform runtime "type replacement" to convert `SupportsFloat` types into `DifferentiableValue`.

In [163]:
def generate_operation(op, self: DifferentiableValue, other: Union[DifferentiableValue, SupportsFloat]):
    if isinstance(other, DifferentiableValue):
        return op(self, other)
    if isinstance(other, (SupportsFloat)):
        return op(self, Constant(other))
    raise TypeError(f"Incompatible type for operation: {type(other)}.")

DifferentiableValue.__add__ = lambda self, other: generate_operation(Sum, self, other)
DifferentiableValue.__sub__ = lambda self, other: self + -other
DifferentiableValue.__neg__ = lambda self: self * -1
DifferentiableValue.__mul__ = lambda self, other: generate_operation(Product, self, other)
DifferentiableValue.__pow__ = lambda self, other: generate_operation(Power, self, other)
DifferentiableValue.__truediv__ = lambda self, other: self * (other ** -1)

Test case:

In [171]:
with Graph() as graph:
    x = Variable(0.458, "x")
    z = Variable(1.45, "x")
    y = Sigmoid(Sine(x))*z
    
print("Raw Graph:")
print(graph.values)
print()

print("Topological Graph:")
topo = generate_topo(y)
for item in topo:
    print(item)
print()

print("Forward value:")
print(graph.forward())
print()
print()

print("Backward graph:")
graph.backward()
print(graph.values)
print()

grad_graph = graph.create_gradient_graph()
print(grad_graph.wrt(x))
del graph

Raw Graph:
[Sine(Variable('x' [0.458], g=0), g=0), Sigmoid(Sine(Variable('x' [0.458], g=0), g=0), g=0), Product(Sigmoid(Sine(Variable('x' [0.458], g=0), g=0), g=0), Variable('x' [1.45], g=0), g=0)]

Topological Graph:
Variable('x' [0.458], g=0)
Sine(Variable('x' [0.458], g=0), g=0)
Sigmoid(Sine(Variable('x' [0.458], g=0), g=0), g=0)
Variable('x' [1.45], g=0)
Product(Sigmoid(Sine(Variable('x' [0.458], g=0), g=0), g=0), Variable('x' [1.45], g=0), g=0)

Forward value:
[[[0.88272003]]]


Backward graph:
[Sine(Variable('x' [0.458], g=[[0.30975273]]), g=[[0.34534441]]), Sigmoid(Sine(Variable('x' [0.458], g=[[0.30975273]]), g=[[0.34534441]]), g=[1.45]), Product(Sigmoid(Sine(Variable('x' [0.458], g=[[0.30975273]]), g=[[0.34534441]]), g=[1.45]), Variable('x' [1.45], g=[[0.60877243]]), g=1)]

[[0.30975273]]
